In [ ]:
!pip install SentenceTransformer

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import pandas as pd
import numpy as np

# Load the SBERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Load the training data into a pandas dataframe
df = pd.read_csv('training_data.csv')

# Create a list of InputExample objects
examples = []
for i, row in df.iterrows():
    examples.append(InputExample(texts=[row['text_a'], row['text_b']], label=row['score']))

# Define the training data generator
def data_generator(examples, batch_size=16):
    for i in range(0, len(examples), batch_size):
        batch = examples[i:i+batch_size]
        texts = [example.texts for example in batch]
        labels = [example.label for example in batch]
        yield texts, labels

# Define the training loss function
train_loss = losses.CosineSimilarityLoss(model)

# Fine-tune the model
model.fit(
    data_generator(examples),
    epochs=5,
    steps_per_epoch=int(np.ceil(len(examples) / 16)),
    warmup_steps=100,
    loss=train_loss,
)

# Save the fine-tuned model
model.save('finetuned_model')
